## Churn Prediction 
### A Machine Learning Model That Can Predict Customers Who Will Leave The Company

The aim is to predict whether a bank's customers leave the bank or not. If the Client has closed his/her bank account, he/she has left.

## Dataset

- **RowNumber:** corresponds to the record (row) number and has no effect on the output.
- **CustomerId:** contains random values and has no effect on customer leaving the bank.
- **Surname:** the surname of a customer has no impact on their decision to leave the bank.
- **CreditScore:** can have an effect on customer churn, since a customer with a higher credit score is less likely to leave the bank.
- **Geography:** a customer’s location can affect their decision to leave the bank.
- **Gender:** it’s interesting to explore whether gender plays a role in a customer leaving the bank.
- **Age:** this is certainly relevant, since older customers are less likely to leave their bank than younger ones.
- **Tenure:** refers to the number of years that the customer has been a client of the bank. Normally, older clients are more loyal and less likely to leave a bank.
- **Balance:** also a very good indicator of customer churn, as people with a higher balance in their accounts are less likely to leave the bank compared to those with lower balances.
- **NumOfProducts:** refers to the number of products that a customer has purchased through the bank.
- **HasCrCard:** denotes whether or not a customer has a credit card. This column is also relevant, since people with a credit card are less likely to leave the bank.
- **IsActiveMember:** active customers are less likely to leave the bank.
- **EstimatedSalary:** as with balance, people with lower salaries are more likely to leave the bank compared to those with higher salaries.
- **Exited:** whether or not the customer left the bank.  (0=No,1=Yes)



### The model created as a result of LightGBM hyperparameter optimization (0.867300)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, \
    classification_report

from scipy.stats import shapiro

import warnings
import missingno as msno

warnings.filterwarnings("ignore")

pd.set_option("display.float_format", lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

low_q1 = 0.05
upper_q3 = 0.95
correlation_limit = 0.60


In [ ]:

def cat_summary(dataframe, categorical_columns, target, plot=False):
    """
    -> Kategorik değişkenlerin sınıflarının oranını ve targettaki medyanı gösterir.

    :param dataframe: İşlem yapılacak dataframe
    :param categorical_columns: Kategorik değişkenlerin adları
    :param target: Dataframe'de ilgilendiğimiz değişken.
    :param plot: Grafik çizdirmek için argüman : True/False

    """
    for col in categorical_columns:
        print(col, " : ", dataframe[col].nunique(), " unique classes.\n")

        print(col, " : ", dataframe[col].value_counts().sum(), "\n")

        print(pd.DataFrame({"COUNT": dataframe[col].value_counts(),
                            "RATIO ( % )": 100 * dataframe[col].value_counts() / len(dataframe),
                            "TARGET_MEDIAN": dataframe.groupby(col)[target].median(),
                            "TARGET_MEAN": dataframe.groupby(col)[target].mean()}), end="\n\n\n")

        if plot:
            sns.countplot(x=col, data=dataframe)

            plt.show()


def hist_for_numeric_columns(dataframe, numeric_columns):
    """
    -> Sayısal değişkenlerin histogramını çizdirir.

    :param dataframe: İşlem yapılacak dataframe.
    :param numeric_columns: Sayısal değişkenlerin adları

    """
    col_counter = 0

    data = dataframe.copy()

    for col in numeric_columns:
        data[col].hist(bins=20)

        plt.xlabel(col)

        plt.title(col)

        plt.show()

        col_counter += 1

    print(col_counter, "variables have been plotted!")


def find_correlation(dataframe, numeric_columns, target, corr_limit=correlation_limit):
    """
    -> Sayısal değişkenlerin targetla olan korelasyonunu inceler.

    :param dataframe: İşlem yapılacak dataframe
    :param numeric_columns: Sayısal değişken adları
    :param target: Korelasyon ilişkisinde bakılacak hedef değişken
    :param corr_limit: Korelasyon sınırı. Sınırdan aşağısı düşük, yukarısı yüksek korelasyon
    :return: İlk değer düşük korelasyona sahip değişkenler, ikinci değer yüksek korelasyona sahip değişkenler
    """
    high_correlations = []

    low_correlations = []

    for col in numeric_columns:
        if col == target:
            pass

        else:
            correlation = dataframe[[col, target]].corr().loc[col, target]

            if abs(correlation) > corr_limit:
                high_correlations.append(col + " : " + str(correlation))

            else:
                low_correlations.append(col + " : " + str(correlation))

    return low_correlations, high_correlations


def outlier_thresholds(dataframe, variable, low_quantile=low_q1, up_quantile=upper_q3):
    """
    -> Verilen değerin alt ve üst aykırı değerlerini hesaplar ve döndürür.

    :param dataframe: İşlem yapılacak dataframe
    :param variable: Aykırı değeri yakalanacak değişkenin adı
    :param low_quantile: Alt eşik değerin hesaplanması için bakılan quantile değeri
    :param up_quantile: Üst eşik değerin hesaplanması için bakılan quantile değeri
    :return: İlk değer olarak verilen değişkenin alt sınır değerini, ikinci değer olarak üst sınır değerini döndürür
    """
    quantile_one = dataframe[variable].quantile(low_quantile)

    quantile_three = dataframe[variable].quantile(up_quantile)

    interquantile_range = quantile_three - quantile_one

    up_limit = quantile_three + 1.5 * interquantile_range

    low_limit = quantile_one - 1.5 * interquantile_range

    return low_limit, up_limit


def has_outliers(dataframe, numeric_columns, plot=False):
    """
    -> Sayısal değişkenlerde aykırı gözlem var mı?

    -> Varsa isteğe göre box plot çizdirme görevini yapar.

    -> Ayrıca aykırı gözleme sahip değişkenlerin ismini göndürür.

    :param dataframe:  İşlem yapılacak dataframe
    :param numeric_columns: Aykırı değerleri bakılacak sayısal değişken adları
    :param plot: Boxplot grafiğini çizdirmek için bool değer alır. True/False
    :return: Aykırı değerlere sahip değişkenlerin adlarını döner
    """
    variable_names = []

    for col in numeric_columns:
        low_limit, up_limit = outlier_thresholds(dataframe, col)

        if dataframe[(dataframe[col] > up_limit) | (dataframe[col] < low_limit)].any(axis=None):
            number_of_outliers = dataframe[(dataframe[col] > up_limit) | (dataframe[col] < low_limit)].shape[0]

            print(col, " : ", number_of_outliers, " aykırı gözlem.")

            variable_names.append(col)

            if plot:
                sns.boxplot(x=dataframe[col])
                plt.show()

    return variable_names


def replace_with_thresholds(dataframe, numeric_columns):
    """
    Baskılama yöntemi

    Silmemenin en iyi alternatifidir.

    Loc kullanıldığından dataframe içinde işlemi uygular.

    :param dataframe: İşlem yapılacak dataframe
    :param numeric_columns: Aykırı değerleri baskılanacak sayısal değişkenlerin adları
    """
    for variable in numeric_columns:
        low_limit, up_limit = outlier_thresholds(dataframe, variable)

        dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit

        dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit





def one_hot_encoder(dataframe, categorical_columns, nan_as_category=False):
    """
    Drop_first doğrusal modellerde yapılması gerekli

    Ağaç modellerde gerekli değil ama yapılabilir.

    dummy_na eksik değerlerden değişken türettirir.

    :param dataframe: İşlem yapılacak dataframe
    :param categorical_columns: One-Hot Encode uygulanacak kategorik değişken adları
    :param nan_as_category: NaN değişken oluştursun mu? True/False
    :return: One-Hot Encode yapılmış dataframe ve bu işlem sonrası oluşan yeni değişken adlarını döndürür.
    """
    original_columns = list(dataframe.columns)

    dataframe = pd.get_dummies(dataframe, columns=categorical_columns,
                               dummy_na=nan_as_category, drop_first=False)

    new_columns = [col for col in dataframe.columns if col not in original_columns]

    return dataframe, new_columns


def rare_analyser(dataframe, categorical_columns, target, rare_perc):
    """
     Data frame değişkenlerinin herhangi bir sınıfı, verilen eşik değerden düşük frekansa sahipse bu değişkenleri gösterir.

    :param dataframe: İşlem yapılacak dataframe
    :param categorical_columns: Rare analizi yapılacak kategorik değişken adları
    :param target: Analizi yapılacak hedef değişken adı
    :param rare_perc: Rare için sınır değer. Altında olanlar rare kategorisine girer.
    :return:
    """
    rare_columns = [col for col in categorical_columns
                    if (dataframe[col].value_counts() / len(dataframe) < rare_perc).any(axis=None)]

    for var in rare_columns:
        print(var, " : ", len(dataframe[var].value_counts()))

        print(pd.DataFrame({"COUNT": dataframe[var].value_counts(),
                            "RATIO": dataframe[var].value_counts() / len(dataframe),
                            "TARGET_MEAN": dataframe.groupby(var)[target].mean(),
                            "TARGET_MEDIAN": dataframe.groupby(var)[target].median()}),
              end="\n\n\n")

    print(len(rare_columns), " adet rare sınıfa sahip değişken var.")



def robust_scaler(variable):
    var_median = variable.median()
    quartile1 = variable.quantile(0.01)
    quartile3 = variable.quantile(0.99)
    interquantile_range = quartile3 - quartile1
    if int(interquantile_range) == 0:
        quartile1 = variable.quantile(0.05)
        quartile3 = variable.quantile(0.95)
        interquantile_range = quartile3 - quartile1
        if int(interquantile_range) == 0:
            quartile1 = variable.quantile(0.25)
            quartile3 = variable.quantile(0.75)
            interquantile_range = quartile3 - quartile1
            z = (variable - var_median) / interquantile_range
            return round(z, 3)

        z = (variable - var_median) / interquantile_range
        return round(z, 3)
    else:
        z = (variable - var_median) / interquantile_range
    return round(z, 3)


In [ ]:
# reading the data
df = pd.read_csv("../input/churn-analysis/churn.csv")

In [ ]:
# Verinin baştan ilk 5 gözlemi
df.head()

In [ ]:
# Verinin sondan ilk 5 gözlemi

df.tail()

In [ ]:
# Kaç farklı müşteri var?
df["CustomerId"].nunique()

In [ ]:
# Anlamsız değişkenlerin düşürülmesi
need_drops = ["RowNumber", "CustomerId", "Surname"]
df.drop(need_drops, axis=1, inplace=True)

In [ ]:
# Eksik gözlem kontrolü
df.isnull().sum()

In [ ]:
# Betimsel istatistiklere bakalım
df.describe([0.01, 0.05, 0.25, 0.50, 0.75, 0.95, 0.99]).T

In [ ]:
# Kategorik değişkenlerin hedefle ilişkisinin incelenmesi
temp_categorical = ["Geography", "Gender", "Tenure", "NumOfProducts", "HasCrCard", "IsActiveMember"]
cat_summary(df, temp_categorical, "Exited")

In [ ]:
# Sayısal değişkenler için histogram incelenmesi
temp_numeric = ["CreditScore", "Age", "Balance", "EstimatedSalary"]
hist_for_numeric_columns(df, temp_numeric)

In [ ]:
# Sayısal değişkenlerin normallik varsayımları
# H0: Normal dağılım varsayımı sağlanmaktadır.
# H1:... sağlanmamaktadır.

# p - value < ise 0.05'ten HO RED.
# p - value < değilse 0.05 H0 REDDEDİLEMEZ.
creditscore = df[["CreditScore"]]
balance = df[["Balance"]]
salary = df[["EstimatedSalary"]]
ages = df[["Age"]]

shapiro(creditscore["CreditScore"])[1] < 0.05

In [ ]:
shapiro(balance["Balance"])[1] < 0.05

In [ ]:
shapiro(salary["EstimatedSalary"])[1] < 0.05

In [ ]:
shapiro(ages["Age"])[1] < 0.05


### Bütün H0'lar reddedildi

In [ ]:
# Korelasyonlara bakalım
low_corr_list, up_corr_list = find_correlation(df, temp_numeric, "Exited")
print("Low Corr List")
for i in low_corr_list:
    print(i)

print("High Corr List")
for i in up_corr_list:
    print(i)

In [ ]:
# Feature Engineering

bins = [15, 25, 40, 55, 100]
names = ['Young', 'Adult', 'Mature', 'Old']
df["NEW_Age_Range"] = pd.cut(df['Age'], bins, labels=names)



bins = [0, 2, 4, 6, 14]
names = ['New', 'Accustomed', 'Loyal', 'Constant']
df["NEW_Tenure_Status"] = pd.cut(df['Tenure'], bins, labels=names)


names = ['CAT1', 'CAT2', 'CAT3', 'CAT4', 'CAT5']
df["NEW_CreditScore_Status"] = pd.qcut(df['CreditScore'], 5, labels=names)

names = ['CAT1', 'CAT2', 'CAT3', 'CAT4', 'CAT5']
df["NEW_EstimatedSalary_Status"] = pd.qcut(df['EstimatedSalary'], 5, labels=names)


df["NEW_Card_Member_Score"] = df["HasCrCard"] * df["IsActiveMember"]

df["NEW_MemberStarts_Age"] = df["Age"] - df["Tenure"]


bins = [5, 25, 40, 55, 100]
names = ['Young', 'Adult', 'Mature', 'Old']
df["NEW_MemberStarts_Age_Range"] = pd.cut(df["NEW_MemberStarts_Age"], bins, labels=names)


In [ ]:

categorical_columns = ["Geography", "Gender", "NumOfProducts",
                       "NEW_Age_Range", "NEW_Tenure_Status",
                       "NEW_CreditScore_Status", "NEW_EstimatedSalary_Status",
                       "NEW_MemberStarts_Age_Range"]

numerical_columns = ["CreditScore", "Age", "Tenure", "Balance",
                     "EstimatedSalary",
                     "NEW_Card_Member_Score", "NEW_MemberStarts_Age"]


rare_analyser(df, categorical_columns, "Exited", 0.5)

In [ ]:
# Nadir sınıflar silinecek
df = df.loc[~((df["NumOfProducts"] == 3) | (df["NumOfProducts"] == 4))]

In [ ]:
# One-Hot Encode
df, one_hot_columns = one_hot_encoder(df, categorical_columns)

In [ ]:
# Robust Scale
need_scale_cols = ["Balance", "EstimatedSalary"]
for col in need_scale_cols:
    df[col] = robust_scaler(df[col])

In [ ]:
df.describe([0.01, 0.05, 0.25, 0.50, 0.75, 0.95, 0.99]).T

In [ ]:
# ****************************************************************#
# Models
X = df.drop("Exited", axis=1)
y = np.ravel(df[["Exited"]])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=123)

rf_params = {"max_depth": [3, 5, 8],
             "max_features": [8, 15, 25],
             "n_estimators": [200, 500, 1000],
             "min_samples_split": [2, 5, 10]}

lgbm_params = {"learning_rate": [0.01, 0.1],
               "n_estimators": [200, 500, 1000],
               "max_depth": [3, 5, 8],
               "colsample_bytree": [1, 0.8, 0.5],
               "num_leaves": [32, 64, 128]}

xgb_params = {"learning_rate": [0.1, 0.01],
              "max_depth": [3, 5, 8],
              "n_estimators": [200, 500, 1000],
              "colsample_bytree": [0.7, 1]}

In [ ]:
rf = RandomForestClassifier(random_state=123)
lgbm = LGBMClassifier(random_state=123)
xgb = XGBClassifier(random_state=123)

In [ ]:
gs_cv_rf = GridSearchCV(rf,
                        rf_params,
                        cv=10,
                        n_jobs=-1,
                        verbose=2).fit(X_train, y_train)

gs_cv_lgbm = GridSearchCV(lgbm,
                          lgbm_params,
                          cv=10,
                          n_jobs=-1,
                          verbose=2).fit(X_train, y_train)

gs_cv_xgb = GridSearchCV(xgb,
                         xgb_params,
                         cv=10,
                         n_jobs=-1,
                         verbose=2).fit(X_train, y_train)

In [ ]:
rf_tuned = RandomForestClassifier(**gs_cv_rf.best_params_, random_state=123).fit(X_train, y_train)

lgbm_tuned = LGBMClassifier(**gs_cv_lgbm.best_params_, random_state=123).fit(X_train, y_train)

xgb_tuned = XGBClassifier(**gs_cv_xgb.best_params_, random_state=123).fit(X_train, y_train)

# Accuracy results
models = [("RF", rf_tuned),
          ("LGBM", lgbm_tuned),
          ("XGB", xgb_tuned)]

for name, model in models:
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    msg = "%s: (%f)" % (name, acc)
    print(msg)